In [2]:
import os, sys
from pathlib import Path
from omegaconf import OmegaConf

# 1) Point this to your project root (the dir that contains `data/`, `models/`, `configs/`, etc.)
PROJECT_ROOT = Path("..").resolve()   # adjust if notebook is nested differently

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)  # optional but often convenient

# 2) Register resolvers
try:
    OmegaConf.register_new_resolver("if", lambda cond, a, b: a if cond else b)
    OmegaConf.register_new_resolver("hydra", lambda _: os.getcwd())
except Exception as e:
    print(f"Error in resolvers: {e}")

In [3]:
import itertools
import numpy as np
import pandas as pd
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.auto import tqdm
import random


import hydra
from omegaconf import OmegaConf

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For more determinism on GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ============================================================
# Simple fusion head (same structure as in MultiTowerFusionV2)
# ============================================================

class FusionHead(nn.Module):
    def __init__(self, fusion_in_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(fusion_in_dim),
            nn.Dropout(dropout),
            nn.Linear(fusion_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 2),
            nn.LogSoftmax(dim=1),  # same as your main model
        )

    def forward(self, x):
        return self.net(x)


# ============================================================
# Grid search trainer for the fusion head only
# ============================================================

class FusionHeadGridSearch:
    """
    1) Uses your FusionDataModule to get train/val loaders.
    2) Uses MultiTowerFusionV2 (with frozen towers) to compute fused features once.
    3) Trains a simple FusionHead on top of precomputed 'fused' vectors.
    4) Performs grid search over (fusion_hidden_dim, lr, n_epochs).
    """

    def __init__(self, cfg, device=None):
        """
        cfg: OmegaConf for your fusion experiment (like fusion_v22.yaml)
        """
        self.cfg = deepcopy(cfg)   # avoid accidental in-place edits
        if device is None:
            device = (
                torch.device("cuda") if torch.cuda.is_available()
                else torch.device("mps") if torch.backends.mps.is_available()
                else torch.device("cpu")
            )
        self.device = device
        print("Using device:", self.device)

        set_seed(self.cfg.seed)

        # Step 1: prepare datamodule and base loaders
        self._init_datamodule()

        # Step 2: build fused features (single pass through towers)
        self._cache_fused_features()

    # ----------------- data & towers -----------------

    def _init_datamodule(self):
        # Instantiate datamodule exactly like in your Trainer
        self.datamodule = hydra.utils.instantiate(self.cfg.datamodule)
        self.datamodule.setup(self.cfg.checkpoint_save_path, type=self.cfg.get("val_type", "random"))

        self.base_train_loader = self.datamodule.train_class_dataloader()
        self.base_val_loader   = self.datamodule.val_dataloader()

        print("Base loaders ready.")
        print("  #train batches:", len(self.base_train_loader))
        print("  #val batches:  ", len(self.base_val_loader))

    def _cache_fused_features(self):
        """
        Use MultiTowerFusionV2 to compute [h_meta | h_tweet | h_desc] for each
        example in train and val, then build frozen DataLoaders returning:
          batch = {"fused": Tensor(B, fusion_in_dim), "label": LongTensor(B)}
        """
        print("\nBuilding towers model for caching fused features...")
        # instantiate MultiTowerFusionV2 (towers frozen)
        model = hydra.utils.instantiate(self.cfg.model.instance).to(self.device)
        model.eval()

        fused_train_list = []
        y_train_list = []

        print("Caching TRAIN fused features...")
        with torch.no_grad():
            for batch in tqdm(self.base_train_loader, desc="Train cache"):
                labels = batch["label"].to(self.device)
                out = model(batch, return_parts=True)

                h_meta  = out["h_meta"]
                h_tweet = out["h_tweet"]
                h_desc  = out["h_desc"]
                fused = torch.cat([h_meta, h_tweet, h_desc], dim=1)  # (B, fusion_in_dim)

                fused_train_list.append(fused.cpu())
                y_train_list.append(labels.cpu())

        fused_train = torch.cat(fused_train_list, dim=0)
        y_train = torch.cat(y_train_list, dim=0)

        fused_val_list = []
        y_val_list = []

        print("Caching VAL fused features...")
        with torch.no_grad():
            for batch in tqdm(self.base_val_loader, desc="Val cache"):
                labels = batch["label"].to(self.device)
                out = model(batch, return_parts=True)

                h_meta  = out["h_meta"]
                h_tweet = out["h_tweet"]
                h_desc  = out["h_desc"]
                fused = torch.cat([h_meta, h_tweet, h_desc], dim=1)

                fused_val_list.append(fused.cpu())
                y_val_list.append(labels.cpu())

        fused_val = torch.cat(fused_val_list, dim=0)
        y_val = torch.cat(y_val_list, dim=0)

        self.fusion_in_dim = fused_train.shape[1]
        print(f"\nFused feature dim = {self.fusion_in_dim}")

        # Build frozen datasets/loaders for fusion head training
        train_ds = TensorDataset(fused_train, y_train)
        val_ds   = TensorDataset(fused_val, y_val)

        batch_size = getattr(self.datamodule, "batch_size", 32)
        num_workers = getattr(self.datamodule, "num_workers", 0)

        self.train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers
        )
        self.val_loader = DataLoader(
            val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )

        print("Cached fused loaders ready.")
        print("  #train samples:", len(train_ds))
        print("  #val samples:  ", len(val_ds))

    # ----------------- single run -----------------

    def _train_single_config(self, hidden_dim, lr, n_epochs, dropout=None):
        """
        Train one FusionHead config on cached fused features.
        Returns: (best_val_auc, best_val_acc, history dict)
        """
        if dropout is None:
            dropout = self.cfg.model.instance.fusion_dropout

        set_seed(self.cfg.seed)  # make runs comparable

        model = FusionHead(
            fusion_in_dim=self.fusion_in_dim,
            hidden_dim=hidden_dim,
            dropout=dropout,
        ).to(self.device)

        criterion = nn.NLLLoss()
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=self.cfg.optim.weight_decay,
        )

        # Optional: cosine scheduler similar to your main code
        total_steps = n_epochs * len(self.train_loader)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=total_steps
        )

        best_val_auc = -np.inf
        best_val_acc = 0.0

        for epoch in range(1, n_epochs + 1):
            # ---- train ----
            model.train()
            train_losses = []

            for fused_batch, y_batch in self.train_loader:
                fused_batch = fused_batch.to(self.device)
                y_batch = y_batch.to(self.device)

                optimizer.zero_grad(set_to_none=True)
                logp = model(fused_batch)  # (B,2)
                loss = criterion(logp, y_batch)
                loss.backward()
                optimizer.step()
                scheduler.step()

                train_losses.append(loss.item())

            # ---- validation ----
            model.eval()
            val_losses = []
            all_probs = []
            all_labels = []

            with torch.no_grad():
                for fused_batch, y_batch in self.val_loader:
                    fused_batch = fused_batch.to(self.device)
                    y_batch = y_batch.to(self.device)

                    logp = model(fused_batch)
                    loss = criterion(logp, y_batch)
                    val_losses.append(loss.item())

                    probs = torch.softmax(logp, dim=1)[:, 1]
                    all_probs.append(probs.cpu())
                    all_labels.append(y_batch.cpu())

            all_probs = torch.cat(all_probs).numpy()
            all_labels = torch.cat(all_labels).numpy()

            preds = (all_probs >= 0.5).astype(int)
            acc = accuracy_score(all_labels, preds)
            try:
                auc = roc_auc_score(all_labels, all_probs)
            except ValueError:
                auc = float("nan")

            mean_train_loss = float(np.mean(train_losses))
            mean_val_loss = float(np.mean(val_losses))

            print(
                f"Epoch {epoch:02d} | "
                f"train_loss={mean_train_loss:.4f} | "
                f"val_loss={mean_val_loss:.4f} | "
                f"val_acc={acc:.4f} | val_auc={auc:.4f}"
            )

            if auc > best_val_auc:
                best_val_auc = auc
                best_val_acc = acc

        return best_val_auc, best_val_acc

    # ----------------- grid search -----------------

    def run_grid_search(
        self,
        hidden_dims,
        lr_list,
        n_epochs_list,
        dropout=None,
    ):
        """
        Brute-force grid over:
          - hidden_dims (fusion_hidden_dim)
          - lr_list
          - n_epochs_list

        Returns:
          results_df: DataFrame with all runs (sorted by AUC, then ACC)
          best_config: dict with best hyperparams
        """
        records = []
        best_auc = -np.inf
        best_config = None

        for hd, lr, n_ep in itertools.product(hidden_dims, lr_list, n_epochs_list):
            print("\n" + "=" * 70)
            print(f"Config: fusion_hidden_dim={hd}, lr={lr}, n_epochs={n_ep}")
            print("=" * 70)

            auc, acc = self._train_single_config(
                hidden_dim=hd,
                lr=lr,
                n_epochs=n_ep,
                dropout=dropout,
            )

            rec = {
                "fusion_hidden_dim": hd,
                "lr": lr,
                "n_epochs": n_ep,
                "val_auc": auc,
                "val_acc": acc,
            }
            records.append(rec)

            if auc > best_auc:
                best_auc = auc
                best_config = rec

        results_df = pd.DataFrame(records).sort_values(
            ["val_acc", "val_auc"], ascending=False
        ).reset_index(drop=True)

        print("\n=== Grid search summary (sorted by AUC, then ACC) ===")
        print(results_df.head(10))
        print("\nBest config:", best_config)

        return results_df, best_config

/users/eleves-a/2023/luis.zuin-ruiz/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
from omegaconf import OmegaConf

os.environ["HYDRA_FULL_ERROR"] = "1"


# Load your base Hydra config
cfg = OmegaConf.load("configs/fusion_v22.yaml")

# (Optional) make sure freeze_towers=True, cache_tower_outputs is irrelevant here
cfg.model.instance.freeze_towers = True

# Instantiate grid search helper
grid_trainer = FusionHeadGridSearch(cfg)

# Define search space
hidden_dims = [64, 128, 256]
lr_list = [1e-5, 1e-4, 2e-4, 5e-4, 1e-3]
n_epochs_list = [1, 2, 3, 4]

# Run grid search
results_df, best_cfg = grid_trainer.run_grid_search(
    hidden_dims=hidden_dims,
    lr_list=lr_list,
    n_epochs_list=n_epochs_list,
    # dropout=0.1  # or leave None to use cfg.model.instance.fusion_dropout
)

Using device: cuda
User-level split:
  #train tweets: 139426
  #val tweets:   15488
  #unique users train: 44158
  #unique users val:   4907
FusionDataModule: label distribution
  Train: label
0    0.534556
1    0.465444
Name: proportion, dtype: float64
  Val: label
0    0.525762
1    0.474238
Name: proportion, dtype: float64
Base loaders ready.
  #train batches: 8715
  #val batches:   968

Building towers model for caching fused features...


Some weights of RobertaModel were not initialized from the model checkpoint at almanach/camembertv2-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at almanach/camembertv2-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Caching TRAIN fused features...


Train cache: 100%|██████████| 8715/8715 [10:36<00:00, 13.69it/s]


Caching VAL fused features...


...

Epoch 02 | train_loss=0.3848 | val_loss=0.3851 | val_acc=0.8286 | val_auc=0.9087


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 03 | train_loss=0.3809 | val_loss=0.3793 | val_acc=0.8334 | val_auc=0.9099


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 04 | train_loss=0.3783 | val_loss=0.3789 | val_acc=0.8330 | val_auc=0.9098

=== Grid search summary (sorted by AUC, then ACC) ===
   fusion_hidden_dim      lr  n_epochs   val_auc   val_acc
0                256  0.0001         4  0.909506  0.834259
1                256  0.0001         3  0.909413  0.833678
2                256  0.0001         2  0.909110  0.833678
3                256  0.0005         2  0.909474  0.833613
4                256  0.0010         4  0.909911  0.833355
5                256  0.0002         2  0.909260  0.833290
6                256  0.0002         3  0.909635  0.833161
7                128  0.0002         3  0.909369  0.833161
8                128  0.0010         3  0.909803  0.833032
9                256  0.0010         1  0.909472  0.833032

Best config: {'fusion_hidden_dim': 256, 'lr': 0.001, 'n_epochs': 4, 'val_auc': 0.9099107286391224, 'val_acc': 0.8333548553719008}


In [5]:
import os
from omegaconf import OmegaConf

os.environ["HYDRA_FULL_ERROR"] = "1"


# Load your base Hydra config
cfg = OmegaConf.load("configs/fusion_v22.yaml")

# (Optional) make sure freeze_towers=True, cache_tower_outputs is irrelevant here
cfg.model.instance.freeze_towers = True

# Instantiate grid search helper
grid_trainer = FusionHeadGridSearch(cfg)

# Define search space
hidden_dims = [256, 512, 1024]
lr_list = [5e-5, 1e-4, 2e-4, 5e-4]
n_epochs_list = [2, 4, 6]

# Run grid search
results_df, best_cfg = grid_trainer.run_grid_search(
    hidden_dims=hidden_dims,
    lr_list=lr_list,
    n_epochs_list=n_epochs_list,
    # dropout=0.1  # or leave None to use cfg.model.instance.fusion_dropout
)

Using device: cuda
User-level split:
  #train tweets: 139426
  #val tweets:   15488
  #unique users train: 44158
  #unique users val:   4907
FusionDataModule: label distribution
  Train: label
0    0.534556
1    0.465444
Name: proportion, dtype: float64
  Val: label
0    0.525762
1    0.474238
Name: proportion, dtype: float64
Base loaders ready.
  #train batches: 8715
  #val batches:   968

Building towers model for caching fused features...


Some weights of RobertaModel were not initialized from the model checkpoint at almanach/camembertv2-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at almanach/camembertv2-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Caching TRAIN fused features...


Train cache:   0%|          | 0/8715 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PA

Caching VAL fused features...


...

Epoch 04 | train_loss=0.3791 | val_loss=0.3809 | val_acc=0.8317 | val_auc=0.9087


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 05 | train_loss=0.3766 | val_loss=0.3790 | val_acc=0.8320 | val_auc=0.9098


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 06 | train_loss=0.3751 | val_loss=0.3790 | val_acc=0.8337 | val_auc=0.9098

=== Grid search summary (sorted by AUC, then ACC) ===
   fusion_hidden_dim       lr  n_epochs   val_auc   val_acc
0                256  0.00010         4  0.909506  0.834259
1                512  0.00005         4  0.909486  0.834130
2               1024  0.00005         6  0.909736  0.834001
3                512  0.00020         4  0.909873  0.833936
4               1024  0.00050         6  0.909839  0.833678
5                512  0.00010         4  0.909746  0.833678
6                256  0.00010         2  0.909110  0.833678
7                256  0.00050         2  0.909474  0.833613
8                256  0.00020         2  0.909260  0.833290
9                512  0.00010         6  0.909789  0.833226

Best config: {'fusion_hidden_dim': 1024, 'lr': 0.0001, 'n_epochs': 4, 'val_auc': 0.9099783507315249, 'val_acc': 0.8331611570247934}


In [5]:
import itertools
import numpy as np
import pandas as pd
from copy import deepcopy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm.auto import tqdm
import random


import hydra
from omegaconf import OmegaConf

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # For more determinism on GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ============================================================
# Simple fusion head (same structure as in MultiTowerFusionV2)
# ============================================================

class FusionHead(nn.Module):
    def __init__(self, fusion_in_dim, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(fusion_in_dim),
            nn.Dropout(dropout),
            nn.Linear(fusion_in_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 2),
            nn.LogSoftmax(dim=1),  # same as your main model
        )

    def forward(self, x):
        return self.net(x)


# ============================================================
# Grid search trainer for the fusion head only
# ============================================================

class FusionHeadGridSearch:
    """
    1) Uses your FusionDataModule to get train/val loaders.
    2) Uses MultiTowerFusionV2 (with frozen towers) to compute fused features once.
    3) Trains a simple FusionHead on top of precomputed 'fused' vectors.
    4) Performs grid search over (fusion_hidden_dim, lr, n_epochs).
    """

    def __init__(self, cfg, device=None):
        """
        cfg: OmegaConf for your fusion experiment (like fusion_v22.yaml)
        """
        self.cfg = deepcopy(cfg)   # avoid accidental in-place edits
        if device is None:
            device = (
                torch.device("cuda") if torch.cuda.is_available()
                else torch.device("mps") if torch.backends.mps.is_available()
                else torch.device("cpu")
            )
        self.device = device
        print("Using device:", self.device)

        set_seed(self.cfg.seed)

        # Step 1: prepare datamodule and base loaders
        self._init_datamodule()

        # Step 2: build fused features (single pass through towers)
        self._cache_fused_features()

    # ----------------- data & towers -----------------

    def _init_datamodule(self):
        # Instantiate datamodule exactly like in your Trainer
        self.datamodule = hydra.utils.instantiate(self.cfg.datamodule)
        self.datamodule.setup(self.cfg.checkpoint_save_path, type=self.cfg.get("val_type", "random"))

        self.base_train_loader = self.datamodule.train_class_dataloader()
        self.base_val_loader   = self.datamodule.val_dataloader()

        print("Base loaders ready.")
        print("  #train batches:", len(self.base_train_loader))
        print("  #val batches:  ", len(self.base_val_loader))

    def _cache_fused_features(self):
        """
        Use MultiTowerFusionV2 to compute [h_meta | h_tweet | h_desc] for each
        example in train and val, then build frozen DataLoaders returning:
          batch = {"fused": Tensor(B, fusion_in_dim), "label": LongTensor(B)}
        """
        print("\nBuilding towers model for caching fused features...")
        # instantiate MultiTowerFusionV2 (towers frozen)
        model = hydra.utils.instantiate(self.cfg.model.instance).to(self.device)
        model.eval()

        fused_train_list = []
        y_train_list = []

        print("Caching TRAIN fused features...")
        with torch.no_grad():
            for batch in tqdm(self.base_train_loader, desc="Train cache"):
                labels = batch["label"].to(self.device)
                out = model(batch, return_parts=True)

                h_meta  = out["h_meta"]
                h_tweet = out["h_tweet"]
                h_desc  = out["h_desc"]
                fused = torch.cat([h_meta, h_tweet, h_desc], dim=1)  # (B, fusion_in_dim)

                fused_train_list.append(fused.cpu())
                y_train_list.append(labels.cpu())

        fused_train = torch.cat(fused_train_list, dim=0)
        y_train = torch.cat(y_train_list, dim=0)

        fused_val_list = []
        y_val_list = []

        print("Caching VAL fused features...")
        with torch.no_grad():
            for batch in tqdm(self.base_val_loader, desc="Val cache"):
                labels = batch["label"].to(self.device)
                out = model(batch, return_parts=True)

                h_meta  = out["h_meta"]
                h_tweet = out["h_tweet"]
                h_desc  = out["h_desc"]
                fused = torch.cat([h_meta, h_tweet, h_desc], dim=1)

                fused_val_list.append(fused.cpu())
                y_val_list.append(labels.cpu())

        fused_val = torch.cat(fused_val_list, dim=0)
        y_val = torch.cat(y_val_list, dim=0)

        self.fusion_in_dim = fused_train.shape[1]
        print(f"\nFused feature dim = {self.fusion_in_dim}")

        # Build frozen datasets/loaders for fusion head training
        train_ds = TensorDataset(fused_train, y_train)
        val_ds   = TensorDataset(fused_val, y_val)

        batch_size = getattr(self.datamodule, "batch_size", 32)
        num_workers = getattr(self.datamodule, "num_workers", 0)

        self.train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers
        )
        self.val_loader = DataLoader(
            val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )

        print("Cached fused loaders ready.")
        print("  #train samples:", len(train_ds))
        print("  #val samples:  ", len(val_ds))

    # ----------------- single run -----------------

    def _train_single_config(self, hidden_dim, lr, n_epochs, dropout=None):
        """
        Train one FusionHead config on cached fused features.
        Returns: (best_val_auc, best_val_acc, history dict)
        """
        if dropout is None:
            dropout = self.cfg.model.instance.fusion_dropout

        set_seed(self.cfg.seed)  # make runs comparable

        model = FusionHead(
            fusion_in_dim=self.fusion_in_dim,
            hidden_dim=hidden_dim,
            dropout=dropout,
        ).to(self.device)

        criterion = nn.NLLLoss()
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=lr,
            weight_decay=self.cfg.optim.weight_decay,
        )

        # Optional: cosine scheduler similar to your main code
        total_steps = n_epochs * len(self.train_loader)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=total_steps
        )

        best_val_auc = -np.inf
        best_val_acc = 0.0

        for epoch in range(1, n_epochs + 1):
            # ---- train ----
            model.train()
            train_losses = []

            for fused_batch, y_batch in self.train_loader:
                fused_batch = fused_batch.to(self.device)
                y_batch = y_batch.to(self.device)

                optimizer.zero_grad(set_to_none=True)
                logp = model(fused_batch)  # (B,2)
                loss = criterion(logp, y_batch)
                loss.backward()
                optimizer.step()
                scheduler.step()

                train_losses.append(loss.item())

            # ---- validation ----
            model.eval()
            val_losses = []
            all_probs = []
            all_labels = []

            with torch.no_grad():
                for fused_batch, y_batch in self.val_loader:
                    fused_batch = fused_batch.to(self.device)
                    y_batch = y_batch.to(self.device)

                    logp = model(fused_batch)
                    loss = criterion(logp, y_batch)
                    val_losses.append(loss.item())

                    probs = torch.softmax(logp, dim=1)[:, 1]
                    all_probs.append(probs.cpu())
                    all_labels.append(y_batch.cpu())

            all_probs = torch.cat(all_probs).numpy()
            all_labels = torch.cat(all_labels).numpy()

            best_t = 0.5
            best_acc = 0.0
            for t in np.linspace(0.0, 1.0, 201):  # step=0.005
                preds_t = (all_probs >= t).astype(int)
                acc_t = accuracy_score(all_labels, preds_t)
                if acc_t > best_acc:
                    best_acc = acc_t
                    best_t = t

            preds = (all_probs >= 0.5).astype(int)
            val_acc = accuracy_score(all_labels, preds)

            print(f"[VAL] default-threshold acc = {val_acc:.4f}, "
                f"best-threshold acc = {best_acc:.4f} @ t={best_t:.3f}")
            acc = best_acc
            try:
                auc = roc_auc_score(all_labels, all_probs)
            except ValueError:
                auc = float("nan")

            mean_train_loss = float(np.mean(train_losses))
            mean_val_loss = float(np.mean(val_losses))

            print(
                f"Epoch {epoch:02d} | "
                f"train_loss={mean_train_loss:.4f} | "
                f"val_loss={mean_val_loss:.4f} | "
                f"val_acc={acc:.4f} | val_auc={auc:.4f}"
            )

            if auc > best_val_auc:
                best_val_auc = auc
                best_val_acc = acc

        return best_val_auc, best_val_acc

    # ----------------- grid search -----------------

    def run_grid_search(
        self,
        hidden_dims,
        lr_list,
        n_epochs_list,
        dropout=None,
    ):
        """
        Brute-force grid over:
          - hidden_dims (fusion_hidden_dim)
          - lr_list
          - n_epochs_list

        Returns:
          results_df: DataFrame with all runs (sorted by AUC, then ACC)
          best_config: dict with best hyperparams
        """
        records = []
        best_auc = -np.inf
        best_config = None

        for hd, lr, n_ep in itertools.product(hidden_dims, lr_list, n_epochs_list):
            print("\n" + "=" * 70)
            print(f"Config: fusion_hidden_dim={hd}, lr={lr}, n_epochs={n_ep}")
            print("=" * 70)

            auc, acc = self._train_single_config(
                hidden_dim=hd,
                lr=lr,
                n_epochs=n_ep,
                dropout=dropout,
            )

            rec = {
                "fusion_hidden_dim": hd,
                "lr": lr,
                "n_epochs": n_ep,
                "val_auc": auc,
                "val_acc": acc,
            }
            records.append(rec)

            if auc > best_auc:
                best_auc = auc
                best_config = rec

        results_df = pd.DataFrame(records).sort_values(
            ["val_acc", "val_auc"], ascending=False
        ).reset_index(drop=True)

        print("\n=== Grid search summary (sorted by AUC, then ACC) ===")
        print(results_df.head(10))
        print("\nBest config:", best_config)

        return results_df, best_config

In [6]:
import os
from omegaconf import OmegaConf

os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "1"


# Load your base Hydra config
cfg = OmegaConf.load("configs/fusion_v22.yaml")

# (Optional) make sure freeze_towers=True, cache_tower_outputs is irrelevant here
cfg.model.instance.freeze_towers = True

# Instantiate grid search helper
grid_trainer = FusionHeadGridSearch(cfg)

# Define search space
hidden_dims = [256, 512]
lr_list = [1e-4, 2e-4, 5e-4, 1e-3]
n_epochs_list = [4]

# Run grid search
results_df, best_cfg = grid_trainer.run_grid_search(
    hidden_dims=hidden_dims,
    lr_list=lr_list,
    n_epochs_list=n_epochs_list,
    # dropout=0.1  # or leave None to use cfg.model.instance.fusion_dropout
)

Using device: cuda
User-level split:
  #train tweets: 139426
  #val tweets:   15488
  #unique users train: 44158
  #unique users val:   4907
FusionDataModule: label distribution
  Train: label
0    0.534556
1    0.465444
Name: proportion, dtype: float64
  Val: label
0    0.525762
1    0.474238
Name: proportion, dtype: float64


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /almanach/camembertv2-base/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f95efe5dfa0>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: a1136961-9825-462d-938b-00febeb285de)')' thrown while requesting HEAD https://huggingface.co/almanach/camembertv2-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Base loaders ready.
  #train batches: 8715
  #val batches:   968

Building towers model for caching fused features...


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /almanach/camembertv2-base/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f95daafe7c0>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: 63ff71ef-0ed0-4881-9b90-06479c353428)')' thrown while requesting HEAD https://huggingface.co/almanach/camembertv2-base/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /almanach/camembertv2-base/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f95daafec70>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: 6a1df10d-d1bd-4392-aac6-7657ad0dffbe)')' thrown while requesting HEAD https://huggingface.co/almanach/ca

Caching TRAIN fused features...


Train cache: 100%|██████████| 8715/8715 [08:38<00:00, 16.81it/s]


Caching VAL fused features...


Val cache: 100%|██████████| 968/968 [01:03<00:00, 15.16it/s]



Fused feature dim = 272
Cached fused loaders ready.
  #train samples: 139426
  #val samples:   15488

Config: fusion_hidden_dim=256, lr=0.0001, n_epochs=4
[VAL] default-threshold acc = 0.8342, best-threshold acc = 0.8342 @ t=0.500
Epoch 01 | train_loss=0.3925 | val_loss=0.3813 | val_acc=0.8342 | val_auc=0.9083
[VAL] default-threshold acc = 0.8326, best-threshold acc = 0.8348 @ t=0.440
Epoch 02 | train_loss=0.3837 | val_loss=0.3804 | val_acc=0.8348 | val_auc=0.9090
[VAL] default-threshold acc = 0.8318, best-threshold acc = 0.8357 @ t=0.410
Epoch 03 | train_loss=0.3810 | val_loss=0.3810 | val_acc=0.8357 | val_auc=0.9094
[VAL] default-threshold acc = 0.8337, best-threshold acc = 0.8356 @ t=0.425
Epoch 04 | train_loss=0.3801 | val_loss=0.3797 | val_acc=0.8356 | val_auc=0.9095

Config: fusion_hidden_dim=256, lr=0.0002, n_epochs=4
[VAL] default-threshold acc = 0.8332, best-threshold acc = 0.8346 @ t=0.460
Epoch 01 | train_loss=0.3905 | val_loss=0.3810 | val_acc=0.8346 | val_auc=0.9085
[VAL]

In [4]:
import os
from omegaconf import OmegaConf

os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "1"


# Load your base Hydra config
cfg = OmegaConf.load("configs/fusion_v22.yaml")

# (Optional) make sure freeze_towers=True, cache_tower_outputs is irrelevant here
cfg.model.instance.freeze_towers = True

# Instantiate grid search helper
grid_trainer = FusionHeadGridSearch(cfg)

# Define search space
hidden_dims = [512, 1024]
lr_list = [1e-4, 1e-3, 1e-2]
n_epochs_list = [2, 4]

# Run grid search
results_df, best_cfg = grid_trainer.run_grid_search(
    hidden_dims=hidden_dims,
    lr_list=lr_list,
    n_epochs_list=n_epochs_list,
    # dropout=0.1  # or leave None to use cfg.model.instance.fusion_dropout
)

Using device: cuda
User-level split:
  #train tweets: 139426
  #val tweets:   15488
  #unique users train: 44158
  #unique users val:   4907
FusionDataModule: label distribution
  Train: label
0    0.534556
1    0.465444
Name: proportion, dtype: float64
  Val: label
0    0.525762
1    0.474238
Name: proportion, dtype: float64
Base loaders ready.
  #train batches: 8715
  #val batches:   968

Building towers model for caching fused features...


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /almanach/camembertv2-base/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f975895cd60>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: c81d4914-0bf3-443f-8dc8-b228ed718f7c)')' thrown while requesting HEAD https://huggingface.co/almanach/camembertv2-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /almanach/camembertv2-base/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f975895ce20>: Failed to resolve \'huggingface.co\' ([Errno -2] Name or service not known)"))'), '(Request ID: e653d723-f534-4a35-bf37-9963f26ca9b9)')' thrown while requesting HEAD https://huggingface.co/almanach/ca

Caching TRAIN fused features...


Train cache: 100%|██████████| 8715/8715 [14:09<00:00, 10.26it/s]


Caching VAL fused features...


Val cache: 100%|██████████| 968/968 [01:15<00:00, 12.91it/s]



Fused feature dim = 272
Cached fused loaders ready.
  #train samples: 139426
  #val samples:   15488

Config: fusion_hidden_dim=512, lr=0.0001, n_epochs=2
Epoch 01 | train_loss=0.3904 | val_loss=0.3822 | val_acc=0.8318 | val_auc=0.9084
Epoch 02 | train_loss=0.3816 | val_loss=0.3804 | val_acc=0.8334 | val_auc=0.9092

Config: fusion_hidden_dim=512, lr=0.0001, n_epochs=4
Epoch 01 | train_loss=0.3907 | val_loss=0.3836 | val_acc=0.8311 | val_auc=0.9077
Epoch 02 | train_loss=0.3824 | val_loss=0.3797 | val_acc=0.8338 | val_auc=0.9094
Epoch 03 | train_loss=0.3799 | val_loss=0.3794 | val_acc=0.8328 | val_auc=0.9095
Epoch 04 | train_loss=0.3793 | val_loss=0.3795 | val_acc=0.8321 | val_auc=0.9096

Config: fusion_hidden_dim=512, lr=0.001, n_epochs=2
Epoch 01 | train_loss=0.3911 | val_loss=0.3844 | val_acc=0.8297 | val_auc=0.9073
Epoch 02 | train_loss=0.3808 | val_loss=0.3799 | val_acc=0.8325 | val_auc=0.9093

Config: fusion_hidden_dim=512, lr=0.001, n_epochs=4
Epoch 01 | train_loss=0.3921 | val_l

KeyboardInterrupt: 